My hypothesis is that players who have to contest a lot of shots outside the paint, particularly threes, have a poor defensive rebounding rate.

Now I want to study 3 things:
- Impact of volume of shots contested inside the arc on DRB rate
- Impact of shots contested outside the arc on DRB rate
- Impact of the ratio of those contests on DRB rate.

Because there are some players who contest a high volume of both. So if you're around the rim a lot, you'll rebound a lot. 

A follow up would be to not do DRB rate - but DRB chances. Actually that might just be a better study

My general overall theory is that when mismatch hunting -don't target the smallest players and weakest defenders. Target the best rim protectors and rebounders. Put small players in positions of help and rebounding instead.

1) You pull their best paint defender and rebounder away from the basket

2) Put weakest defenders in help spots

3) Capitalize on OREBs

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashptdefend, leaguedashptstats

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

data_DIR = "../data/misc/"
box_DIR = "../data/box/"

In [ ]:
def_cat = ['Less Than 6Ft','Less Than 10Ft','Greater Than 15Ft','3 Pointers']
col_names =["FGA_LT_06","FGA_LT_10","FGA_GT_15","FG3A"]

In [ ]:
dfs = []
for year in tqdm(range(2013,2024)):
    season = get_ss(year)
    dfa = []
    for i in range(len(def_cat)):
        stats = leaguedashptdefend.LeagueDashPtDefend(
            defense_category=def_cat[i],
            league_id="00",
            per_mode_simple="Totals",
            season=season,
            season_type_all_star="Regular Season",
        )
        df = stats.get_data_frames()[0]
        df = df.rename(columns={"CLOSE_DEF_PERSON_ID":"pID","PLAYER_NAME":"player"})
        df = df[["pID","player",col_names[i]]]
        dfa.append(df)
        time.sleep(0.6)
    dfd = pd.merge(dfa[0],dfa[1])
    for i in range(2,4):
        dfd = pd.merge(dfd,dfa[i])
    stats = leaguedashptstats.LeagueDashPtStats(
        per_mode_simple="Totals",
        player_or_team="Player",
        pt_measure_type="Rebounding",
        season=season,
        season_type_all_star="Regular Season",
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.rename(columns={"PLAYER_ID":"pID"})
    df1 = df1[["pID","DREB","DREB_CHANCES","DREB_CHANCE_PCT"]]
    df2 = pd.merge(dfd,df1)
    df2["season"] = year + 1
    dfs.append(df2)
    time.sleep(0.6)

In [ ]:
dfb = pd.concat(dfs)

In [ ]:
dfb.to_csv(data_DIR + "to_shamit_totals.csv",index=False)

In [ ]:
dfc = dfb.query("DREB_CHANCES > 100").query("FGA_LT_10 > 100")

In [ ]:
dfc["Diff_LT10_3"] = dfc["FGA_LT_10"] - dfc["FG3A"]

In [ ]:
x_p = dfc["Diff_LT10_3"]
y_p = dfc["DREB_CHANCES"]
slope, intercept, r, p, sterr = scipy.stats.linregress(x=x_p, y=y_p)
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=x_p, y=y_p, color="black", scatter_kws={"color": "tab:blue"}, ax=ax
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
plt.show()

In [ ]:
dfs = []
for year in tqdm(range(2013,2024)):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_P_Hustle_{year}.parquet",columns=['gameId','teamTricode','personId', 'firstName', 'familyName','contestedShots2pt', 'contestedShots3pt'])
    df2 = pd.read_parquet(box_DIR + f"NBA_Box_P_Track_{year}.parquet",columns=['gameId','personId','reboundChancesDefensive','defendedAtRimFieldGoalsAttempted'])
    df3 = pd.merge(df1,df2)
    df3['season'] = year + 1
    dfs.append(df3)
df = pd.concat(dfs)

In [ ]:
df.to_csv(data_DIR + "to_shamit_boxscores_contest.csv",index=False)

In [ ]:
dfp = df.query("personId == 203076")

In [ ]:
x_p = dfp["defendedAtRimFieldGoalsAttempted"]
y_p = dfp["reboundChancesDefensive"]
slope, intercept, r, p, sterr = scipy.stats.linregress(x=x_p, y=y_p)
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=x_p, y=y_p, color="black", scatter_kws={"color": "tab:blue"}, ax=ax
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
plt.show()

In [ ]:
dfp